In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer

In [26]:
housing = pd.read_csv('../Datasets/orignial_ames_housingprices.csv')
housing.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,...,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageArea,PavedDrive,SalePrice
0,60,RL,65.0,8450,CollgCr,1Fam,2Story,7,5,2003,...,1710,1,0,2,1,3,0,548,Y,208500
1,20,RL,80.0,9600,Veenker,1Fam,1Story,6,8,1976,...,1262,0,1,2,0,3,1,460,Y,181500
2,60,RL,68.0,11250,CollgCr,1Fam,2Story,7,5,2001,...,1786,1,0,2,1,3,1,608,Y,223500
3,70,RL,60.0,9550,Crawfor,1Fam,2Story,7,5,1915,...,1717,1,0,1,0,3,1,642,Y,140000
4,60,RL,84.0,14260,NoRidge,1Fam,2Story,8,5,2000,...,2198,1,0,2,1,4,1,836,Y,250000


In [27]:
kidney = pd.read_csv('../Datasets/kidney_disease.csv')
kidney.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,hemo,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48,80,1.020,1,0,?,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.020,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.010,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.010,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [43]:
kidney.replace('?', np.NaN, inplace=True)

In [82]:
X = kidney.drop('classification', axis=1)
y = kidney['classification']

In [83]:
pip install sklearn-pandas==2.2.0

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install feature_engine

Note: you may need to restart the kernel to use updated packages.


In [80]:
from sklearn_pandas import DataFrameMapper
from feature_engine.imputation import CategoricalImputer
from sklearn.impute import SimpleImputer

In [84]:
X.isnull().sum()

id         9
age       12
bp        47
sg        46
al        49
su       152
rbc       65
pc         4
pcc        4
ba        44
bgr       19
bu        17
sc        87
sod       88
pot       52
hemo      71
wc       106
rc       131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64

In [95]:
cat_feature1 = X.dtypes == object

In [96]:
cat_feature2 = X.columns[cat_feature1].tolist()
cat_feature2[:3]

['id', 'age', 'bp']

In [97]:
non_cat = X.columns[~cat_feature1].tolist()

In [98]:
num_imp_mapper = DataFrameMapper([([f], SimpleImputer(strategy='median')) for f in non_cat], input_df=True, df_out=True)

In [99]:
cat_imp_mapper = DataFrameMapper([([d], CategoricalImputer()) for d in cat_feature2], input_df=True, df_out=True)

In [100]:
from sklearn.pipeline import FeatureUnion

In [106]:
union = FeatureUnion([('num_map', num_imp_mapper), ('cat_map', cat_imp_mapper)])

In [107]:
from sklearn.base import BaseEstimator, TransformerMixin

class Dictifier(BaseEstimator, TransformerMixin):
    def fit(self, X,y=None):
        return self
    def transform(self,X):
        if type(X) == pd.core.frame.DataFrame:
            return X.to_dict('records')
        else:
            return pd.DataFrame(X).to_dict('records')

In [105]:
pipeline2 = Pipeline([('union', union ), ('dictifier', Dictifier()), ('vectorizer', DictVectorizer(sort=False)), ('xgb', xgb.XGBClassifier(max_depth=3))])

In [104]:
cv = cross_val_score(pipeline2, X,y, scoring='roc_auc', cv=3)

C:\Users\l_san\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
3 fits failed out of a total of 3.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\l_san\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\l_san\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\l_san\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 355, in _fit
    **fit_param